## Quiz 2: Funciones heurísticas para el 8-puzzle

Adaptado de Russell & Norvig (2016), sec. 3.6.

### 8-puzzle

El siguiente problema se conoce como el 8-puzzle. En un tablero $3\times 3$ se ponen ocho fichas, cada una con un número del 1 al 8, dejando un espacio vacío. Partiendo de una configuración aleatoria, el objetivo es encontrar una secuencia de desplazamientos de fichas al espacio vacío hasta llegar a un tablero ordenado, como se muestra en la figura siguiente:


<img src="./imagenes/8-puzzle.png" width="300">

**Ejercicio 1:**

Proporcione la descripción formal del problema:

* **Estado inicial**: 

* **Posibles acciones**: 

* **Función de transiciones**: 

* **Prueba de satisfacción del objetivo**: 

* **Función de costo**: 

----


**Implementación del problema**

Implementaremos el problema del 8-puzzle.

In [1]:
import numpy as np
from random import choice
import copy

In [38]:
class ocho_puzzle:
    
    inicial = list(np.random.choice(9, 9, replace=False))
    
    def estado_inicial(self):
        estado = np.reshape(self.inicial, (3,3))
        return estado
    
    def acciones_aplicables(self, estado):
        # Devuelve una lista de fichas que es posible mover
        # y en qué dirección
        # Input: estado, que es una np.matrix(8x8)
        # Output: lista de parejas ((x1,y1), (x2,y2))
        # Es decir, la ficha en la posición (x1,y1) puede moverse a (x2,y2)
        y, x = np.where(estado == 0)
        y = y[0]
        x = x[0]
        if x == 0:
            if y == 0:
                return [((x + 1, y), (x, y)), 
                        ((x, y + 1), (x, y))
                       ]
            elif y == 2:
                return [((x + 1, y), (x, y)), 
                        ((x, y - 1), (x, y))
                       ]
            else:
                return [((x + 1, y), (x, y)), 
                        ((x, y + 1), (x, y)),
                        ((x, y - 1), (x, y))
                       ]
        if x == 2:
            if y == 0:
                return [((x - 1, y), (x, y)), 
                        ((x, y + 1), (x, y))
                       ]
            elif y == 2:
                return [((x - 1, y), (x, y)), 
                        ((x, y - 1), (x, y))
                       ]
            else:
                return [((x - 1, y), (x, y)), 
                        ((x, y + 1), (x, y)),
                        ((x, y - 1), (x, y))
                       ]
        else:
            if y == 0:
                return [((x - 1, y), (x, y)),
                        ((x + 1, y), (x, y)),
                        ((x, y + 1), (x, y))
                       ]
            elif y == 2:
                return [((x - 1, y), (x, y)),
                        ((x + 1, y), (x, y)),
                        ((x, y - 1), (x, y))
                       ]
            else:
                return [((x - 1, y), (x, y)), 
                        ((x + 1, y), (x, y)),
                        ((x, y + 1), (x, y)),
                        ((x, y - 1), (x, y))
                       ]

    def transicion(self, estado, indices):
        # Devuelve el tablero moviendo la ficha en indice
        # Input: estado, que es una np.matrix(8x8)
        #        indice, de la forma ((x1,y1), (x2,y2))
        # Output: estado, que es una np.matrix(8x8)
        
        s = copy.deepcopy(estado)
        x1, y1 = indices[0]
        x2, y2 = indices[1]
        s[y2, x2] = estado[y1, x1]
        s[y1, x1] = 0
        return s
    
    def test_objetivo(self, estado):
        # Devuelve True/False dependiendo si el estado
        # resuelve el problema
        # Input: estado, que es una np.matrix(8x8)
        # Output: True/False
        k = list(np.reshape(estado, (9,1)))
        k = [x[0] for x in k]
        return (k == list(range(9)))
        
    def costo(self, estado, accion):
        return 1

In [39]:
P = ocho_puzzle()
s1 = P.estado_inicial()
print("Estado inicial:\n", s1)
print("Objetivo?:", P.test_objetivo(s1))
s = np.reshape(range(9), (3,3))
print("Estado objetivo:\n", s)
print("Objetivo?:", P.test_objetivo(s))

Estado inicial:
 [[2 0 8]
 [4 7 6]
 [1 3 5]]
Objetivo?: False
Estado objetivo:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
Objetivo?: True


### Funciones heurísticas

Heurística es una palabra que viene del griego εὑρίσκειν, y que significa "hallar" o "inventar". Una función heurística es una manera de usar conocimiento sobre el problema (*domain knowledge*) para buscar una solución de manera más eficiente que las estrategias no informadas.

Para el 8-puzzle se han encontrado dos funciones con muy buenos resultados:

- $h_1 = $ número de fichas que no corresponden al orden del estado objetivo.

- $h_2 = $ suma de la distancia de cada ficha a su lugar en el estado objetivo, usando la distancia Manhattan, también conocida como la [distancia del taxista](https://es.wikipedia.org/wiki/Geometr%C3%ADa_del_taxista). 

**Ejercicio 2:**

Implemente las dos funciones heurísticas, $h_1$ y $h_2$, para el 8-puzzle. 

**Respuesta:**

Una posible implementación es la siguiente:

In [40]:
def h1(estado):
    distancia = 0
    for i in range(1, 3 * 3):
        x = i % 3
        y = int(i / 3)
        if estado[y, x] != i:
            distancia += 1

    return distancia

def h2(estado):
    distancia = 0
    final = np.reshape(range(9), (3,3))
    for i in range(1, 9):
        y1, x1 = np.where(estado == i)
        y2, x2 = np.where(final == i)
        distancia += np.abs(x1 - x2) + np.abs(y1 - y2)

    return distancia[0]

In [41]:
P = ocho_puzzle()
s = P.estado_inicial()
print("s\n", s)
print("h1(s) =", h1(s))
print("h2(s) =", h2(s))

s
 [[2 0 8]
 [4 7 6]
 [1 3 5]]
h1(s) = 8
h2(s) = 15


---

**Ejercicio 3:**

Lea la sección 3.5.1 del texto guía e implemente el algoritmo *greedy best-first search*. Use este algoritmo para encontrar una solución al 8-puzzle.

**Respuesta:**

Una posible implementación es la siguiente:

In [42]:
# Para poder verificar si un estado ya ha sido considerado
def codifica_estado(estado):
    valores = [str(estado[y, x]) for y in range(3) for x in range(3)]
    s = ''.join(str(i) for i in valores)
    return s

s = P.estado_inicial()
print(s)
print(codifica_estado(s))

[[2 0 8]
 [4 7 6]
 [1 3 5]]
208476135


In [62]:
class nodo:
    
    # Clase para crear los nodos
    
    def __init__(self, estado, madre, accion, costo):
        self.estado = estado
        self.madre = madre
        self.accion = accion
        self.costo = costo
        
def nodo_hijo(problema, madre, accion):
    
    # Función para crear un nuevo nodo
    # Input: problema, que es un objeto de clase ocho_reinas
    #        madre, que es un nodo,
    #        accion, que es una acción que da lugar al estado del nuevo nodo
    # Output: nodo

    return nodo(problema.transicion(madre.estado, accion),
                madre,
                accion,
                costo = madre.costo + problema.costo(madre.estado, accion)
               )

def solucion(n):
    
    # Devuelve la secuencia de estados que va desde la raíz
    # hasta el nodo n
    # Input: n, nodo
    # Output: l, lista de acciones
    
    acciones_invertidas = []
    m = copy.deepcopy(n)
    while m.madre != None:
        acciones_invertidas.append(m.accion)
        m = m.madre
    
    num_acciones = len(acciones_invertidas)
    acciones = []
    for i in range(1, num_acciones + 1):
        acciones.append(acciones_invertidas[num_acciones - i])
    
    return acciones

def tree_search(problema, distancia):
    
    # Función de búsqueda mediante la construcción
    # del arbol de búsqueda de manera aleatoria
    
    raiz = nodo(problema.estado_inicial(), None, None, 0)
#    print("Estado inicial:\n", raiz.estado)
    
    if problema.test_objetivo(raiz.estado):
            return raiz
    else:
        frontera = {distancia(raiz.estado):[raiz]}
        explored = []
#        print("Frontera:", frontera.keys())
#        print("Explored:", explored)

    while (len(frontera) > 0):# and (len(explored) < 4):
        n = choice(frontera[min(frontera.keys())]) # Seleccionamos un nodo aleatorio
#        print("Estado seleccionado:\n", n.estado)
        frontera[distancia(n.estado)].remove(n)
        if len(frontera[distancia(n.estado)]) == 0:
            del frontera[distancia(n.estado)]
        explored.append(codifica_estado(n.estado))
#        print("Frontera:", frontera.keys())
#        print("Explored:", explored)
        acciones = problema.acciones_aplicables(n.estado)
        for a in acciones:
            N = nodo_hijo(problema, n, a)
#            print("Nuevo estado:\n", N.estado)
            if problema.test_objetivo(N.estado):
                return N
            elif codifica_estado(N.estado) not in explored:                
                try:
                    frontera[distancia(N.estado)].append(N)
                except:
                    frontera[distancia(N.estado)] = [N]
    
    return None

In [58]:
r = tree_search(P, h2)
print("Estado encontrado:\n", r.estado)

Estado encontrado:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]


In [65]:
if r:
    l = solucion(r)
    print("La solución tiene " + str(len(l)) + " pasos.")
    print("La solución encontrada es:")
#    print(l)
    s = P.estado_inicial()
    for a in l:
        print("\n", s)
        s = P.transicion(s, a)
    print("\n", s)
else:
    "No se encontró solución"

La solución tiene 37 pasos.
La solución encontrada es:

 [[2 0 8]
 [4 7 6]
 [1 3 5]]

 [[0 2 8]
 [4 7 6]
 [1 3 5]]

 [[4 2 8]
 [0 7 6]
 [1 3 5]]

 [[4 2 8]
 [1 7 6]
 [0 3 5]]

 [[4 2 8]
 [1 7 6]
 [3 0 5]]

 [[4 2 8]
 [1 0 6]
 [3 7 5]]

 [[4 2 8]
 [1 6 0]
 [3 7 5]]

 [[4 2 0]
 [1 6 8]
 [3 7 5]]

 [[4 0 2]
 [1 6 8]
 [3 7 5]]

 [[0 4 2]
 [1 6 8]
 [3 7 5]]

 [[1 4 2]
 [0 6 8]
 [3 7 5]]

 [[1 4 2]
 [3 6 8]
 [0 7 5]]

 [[1 4 2]
 [3 6 8]
 [7 0 5]]

 [[1 4 2]
 [3 0 8]
 [7 6 5]]

 [[1 0 2]
 [3 4 8]
 [7 6 5]]

 [[0 1 2]
 [3 4 8]
 [7 6 5]]

 [[3 1 2]
 [0 4 8]
 [7 6 5]]

 [[3 1 2]
 [7 4 8]
 [0 6 5]]

 [[3 1 2]
 [7 4 8]
 [6 0 5]]

 [[3 1 2]
 [7 4 8]
 [6 5 0]]

 [[3 1 2]
 [7 4 0]
 [6 5 8]]

 [[3 1 2]
 [7 0 4]
 [6 5 8]]

 [[3 1 2]
 [7 5 4]
 [6 0 8]]

 [[3 1 2]
 [7 5 4]
 [6 8 0]]

 [[3 1 2]
 [7 5 0]
 [6 8 4]]

 [[3 1 2]
 [7 0 5]
 [6 8 4]]

 [[3 1 2]
 [0 7 5]
 [6 8 4]]

 [[0 1 2]
 [3 7 5]
 [6 8 4]]

 [[1 0 2]
 [3 7 5]
 [6 8 4]]

 [[1 2 0]
 [3 7 5]
 [6 8 4]]

 [[1 2 5]
 [3 7 0]
 [6 8 4]]

 [[1 2 5]
 [3 

---

### En este notebook usted aprendió

* El concepto de función heurística en la búsqueda de soluciones para implementar *domain knowledge*.

* Implementar el método de búsqueda *greedy best-first search*.